# Metodología

## Importamos la base de datos

Primeramente, importamos las librerías necesarias para la sección.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DB = pd.read_csv('Mall_Customers.csv')

print('Cantidad de datos por atributo =', str(len(DB['Age'])) + '.')
DB.head()

### Normalizamos los datos

Utilizamos la función de normalización generada con anterioridad, agregando además sus parámetros necesarios.

In [ ]:
def Normaliza(DB):
    DB = DB.to_numpy()

    Atributos = DB[0]
    NoAtributos = len(Atributos)
    Instancias = DB.T[0]
    NoInstancias = len(Instancias)

    MaximoDeAtributos = []
    MinimoDeAtributos = []
    for idx, element in enumerate(Atributos):
      CaractMax = max(DB.T[idx])
      CaractMin = min(DB.T[idx])
      MaximoDeAtributos.append(CaractMax)
      MinimoDeAtributos.append(CaractMin)

    DBNorm = []
    MaximoNormalizado = 1
    MinimoNormalizado = 0
    RangoNormalizado = MaximoNormalizado - MinimoNormalizado
    for idx, element in enumerate(Atributos):
      CaractNorm = []
      if str(type(Atributos[idx]))[8 : -2] != 'str':
        RangodeDatos = MaximoDeAtributos[idx] - MinimoDeAtributos[idx]
        for idx2, element2 in enumerate(Instancias):
          if str(DB.T[idx][idx2]) != 'nan':
            D = DB.T[idx][idx2] - MinimoDeAtributos[idx]
            DPct = D / RangodeDatos
            dNorm = RangoNormalizado * DPct
            Normalizado = MinimoNormalizado + dNorm
            CaractNorm.append(Normalizado)
          else:
            CaractNorm.append(DB.T[idx][idx2])
      else:
        for idx2, element2 in enumerate(Instancias):
          CaractNorm.append(DB.T[idx][idx2])
      DBNorm.append(CaractNorm)
    return(DBNorm)

In [ ]:
DB_Norm = Normaliza(DB)

Definimos los valores de X con los datos de ingresos anuales, y los valores de Y con los datos de edad.

In [ ]:
X = np.array(DB_Norm[3])
Y = np.array(DB_Norm[2])

### Visualizamos los datos

In [ ]:
plt.scatter(X, Y, color = 'lightblue', label = 'Datos')
plt.title('Ingreso Anual (k$) vs Edad')
plt.xlabel('Ingreso Anual (k$)')
plt.ylabel('Edad')
plt.legend()
plt.show()

Función para transformar los datos de entrada X e Y a puntos de dato con coordenadas (Xi, Yi).

In [ ]:
def data2point(X, Y):
    puntos = []
    for idx, x in enumerate(X):
        puntos.append((x, Y[idx]))
    return(puntos)

Transformamos los datos X e Y a puntos de dato con coordenadas (Xi, Yi).

In [ ]:
puntos = data2point(X, Y)

### Método de codo

Lo utilizaremos para decidir cuántos grupos (clúster) son necesarios.

Función para calcular las distancias de cada punto hacia cada uno los centroides.

Generando un cúmulo de valores, correspondientes a cada punto, por cada centroide en la lista resultante.

In [ ]:
def distACentrsXpunto(Centrs, DaXpuntos):
    dist = []
    for element in Centrs:
        distXData = []
        for DaXpunto in DaXpuntos:
            distData = ((element[0] - DaXpunto[0])**2 + (element[1] - DaXpunto[1])**2)**0.5
            distXData.append(distData)
        dist.append(distXData)
    return(dist)

Función para generar n centroides que estén lo más alejados posibles entre ellos en un rango entre 0 y 1.

In [ ]:
def CentrsPP(NumDcentrs, PuntosIn):
    First_Centr = (rd.random(), rd.random())
    
    Centrs = [First_Centr]
    for NumCent in range(1, NumDcentrs, 1):
        Dists = distACentrsXpunto(Centrs, PuntosIn)
        Minpunto = Dists[0]
        for CentN in Dists:
            for idx, distN in enumerate(CentN):
                if distN < Minpunto[idx]:
                    Minpunto[idx]  = distN
        distmin = max(Minpunto)
        Minpunto = np.array(Minpunto)
        posMinpunto = np.where(Minpunto == distmin)
        Centrs.append(PuntosIn[posMinpunto[0][0]])
    
    return(Centrs)

Generamos 9 centroides que estén lo más alejado es¿ntre ellos posible, a través de la generación de centroides para k-means++ que se muestra en su respectiva sección más adelante.

In [ ]:
Centrs = CentrsPP(9, puntos)

Analizamos gráficamente los centroides generados.

In [ ]:
Xp = []
Yp = []
for element in Centrs:
    Xp.append(element[0])
    Yp.append(element[1])

plt.scatter(X, Y, color = 'lightblue', label = 'Datos')
plt.scatter(Xp, Yp, color = 'red', label = 'Centroides')
plt.title('Ingreso Anual (k$) vs Edad')
plt.xlabel('Ingreso Anual (k$)')
plt.ylabel('Edad')
plt.legend()
plt.show()

Utilizamos los centroides generados para generar 9 iteraciones, en las cuales cada una contiene de 1 a 9 centroides generados, siendo el número de iteración la cantidad de centroides que contiene.

In [ ]:
idxCentrs = len(Centrs)

Centrs4KM = []
for idxCentr in range(1, (idxCentrs+1)):
    Centrs4KM_par = []
    for idx, K in enumerate(range(1, idxCentr + 1)):
        Centrs4KM_par.append(Centrs[idx])
    Centrs4KM.append(Centrs4KM_par)

Función para determinar a qué centroide corresponde cada uno de los puntos (datos), según la distancia previamente calculada, hacia cada centroide.

In [ ]:
dist_glob = []
for Centr in Centrs4KM:
    dist = distACentrsXpunto(Centr, puntos)
    dist_glob.append(dist)

Calculamos la distancia de cada punto de dato hacia cada uno de los diferentes centroides en cada iteración, los cuales tienen distinta cantidad de centroides.

In [ ]:
def Dato2CentrX(D2Xpuntos, dist):
    MinimIdx = []
    for idx, D2Xpunto in enumerate(D2Xpuntos):
        inim = []
        for element in dist:
            inim.append(element[idx])
        minimo = min(inim)
        MinIdx = inim.index(minimo)
        MinimIdx.append(MinIdx)
    return(MinimIdx)

En base a las distancias calculadas, en cada iteración ubicamos a qué centroide corresponde cada dato. 

In [ ]:
minimf_glob = []
for dist in dist_glob:
    MinimIdx = Dato2CentrX(puntos, dist)
    minimf_glob.append(MinimIdx)

Transformamos la lista que contiene a qué centroide corresponde cada dato en cada una de las iteraciones, en una matriz. Esto con el fin de poder utilizar la funcion de numpy.where.

In [ ]:
minimf_glob = np.array(minimf_glob)

Generamos gráficamente el gráfico tipo codo, para ubicar el valor de K óptimo.

In [ ]:
WCSS = []
for idx1, Centr in enumerate(Centrs4KM):
    sumD = 0
    for idx2, K in enumerate(Centr):
        pos = np.where(minimf_glob[idx1] == idx2)
        for element in pos[0]:
            sumD += (((puntos[element][0] - K[0])**2 + (puntos[element][1] - K[1])**2)**0.5)**2
    WCSS.append(sumD)

idxClusts  = idxCentrs = len(Centrs)
    
plt.plot(range(1, (idxClusts+1)), WCSS, color = 'lightblue', label = 'WCSS según la cantidad de K considerados', linewidth = 3)
plt.scatter(5, WCSS[4], color = 'red', linewidth = 5)
plt.title('Número de clúster vs WCSS')
plt.xlabel('Número de clúster (K)')
plt.ylabel('Within-Cluster Sum-of-Squares (WCSS)')
plt.legend()
plt.show()

De acuerdo a lo observado, se define que K = 5 es el número óptimo de clústers a considerar. 

Sin embargo, al realizar varias pruebas, nos podemos percatar que es muy variable el K óptimo, según los centroides inciales, oscilando entre 3 y 6. Esto, debido a que los centroides son generados de forma aleatoria.

## K-means ++

Primeramente, importamos las librerías necesarias para la sección.

In [ ]:
import random as rd
import matplotlib.pyplot as plt
import numpy as np
from IPython import display
from sklearn.cluster import KMeans

### Funciones propias

Función para generar n centroides que estén lo más alejados posibles entre ellos en un rango entre 0 y 1.

In [ ]:
def CentrsPP(NumDcentrs, PuntosIn):
    First_Centr = (rd.random(), rd.random())
    
    Centrs = [First_Centr]
    for NumCent in range(1, NumDcentrs, 1):
        Dists = distACentrsXpunto(Centrs, PuntosIn)
        Minpunto = Dists[0]
        for CentN in Dists:
            for idx, distN in enumerate(CentN):
                if distN < Minpunto[idx]:
                    Minpunto[idx]  = distN
        distmin = max(Minpunto)
        Minpunto = np.array(Minpunto)
        posMinpunto = np.where(Minpunto == distmin)
        Centrs.append(PuntosIn[posMinpunto[0][0]])
    
    return(Centrs)

### Centroides iniciales

Seguimos el valor de k = 5 obtenido en el método del codo en la sección anterior para generar 3 centroides lo más alejado posibles entre ellos.

In [ ]:
Centroides = CentrsPP(5, puntos)

Mostramos visualmente dónde se encuntran los centroides recién generados.

In [ ]:
Xp = []
Yp = []
for element in Centroides:
    Xp.append(element[0])
    Yp.append(element[1])

plt.scatter(X, Y, color = 'lightblue', label = 'Datos')
plt.scatter(Xp, Yp, color = 'red', label = 'Centroides')
plt.title('Ingreso Anual (k$) vs Edad')
plt.xlabel('Ingreso Anual (k$)')
plt.ylabel('Edad')
plt.legend()
plt.show()

### Funciones propias de K-means común.

Función para generar n centroides aleatoriamente, en forma de lista.

Para valores de (X, Y) normalizados entre 0 y 1

In [ ]:
def Centrs_aleat(NumDCentrs):
    idxCentrs = range(1, NumDCentrs + 1)

    Centrs = []
    for K in idxCentrs:
        Centrs.append((rd.random(), rd.random()))
    return(Centrs)

Función para generar n centroides mediante el promedio de los datos pertenecientes al clúster de cada centroide a generar.

In [ ]:
def Centrs_promedio(Datos):
    Centrs = []
    for K in Datos:
        sumaX = 0
        sumaY = 0
        for data in K:
            sumaX += data[0]
            sumaY += data[1]
        X = sumaX / len(K)
        Y = sumaY / len(K)
        Centrs.append((X, Y))
    return(Centrs)

Función para calcular las distancias de cada punto hacia cada uno los centroides.

Generando un cúmulo de valores, correspondientes a cada punto, por cada centroide en la lista resultante.

In [ ]:
def distACentrsXpunto(Centrs, DaXpuntos):
    dist = []
    for element in Centrs:
        distXData = []
        for DaXpunto in DaXpuntos:
            distData = ((element[0] - DaXpunto[0])**2 + (element[1] - DaXpunto[1])**2)**0.5
            distXData.append(distData)
        dist.append(distXData)
    return(dist)

Función para determinar a qué centroide corresponde cada uno de los puntos (datos), según la distancia previamente calculada, hacia cada centroide.

In [ ]:
def Dato2CentrX(D2Xpuntos, dist):
    MinimIdx = []
    for idx, D2Xpunto in enumerate(D2Xpuntos):
        inim = []
        for element in dist:
            inim.append(element[idx])
        minimo = min(inim)
        MinIdx = inim.index(minimo)
        MinimIdx.append(MinIdx)
    return(MinimIdx)

Función para agrupar cada dato con el resto de datos que pertenecen a un mismo clúster.

In [ ]:
def AgrupDatos(Centroides, CentroideXDato, ADPuntos):
    PuntosDCentr = []
    for idx, Centroide in enumerate(Centroides):
        PuntosDColorX = []
        positions = np.where(CentroideXDato == idx)
        for position in positions[0]:
            PuntosDColorX.append(ADPuntos[position])
        PuntosDCentr.append(PuntosDColorX)
    return(PuntosDCentr)

Función para predecir a qué clúster pertenecen nuevos datos ingresados en la forma [(X1,Y1),...,(Xn,  Yn)].

Elige el clúster con el centroide más cercano al dato ingresado.

In [ ]:
def Predict(data):
    distancias = distACentrsXpunto(Centroides, data)
    CentroideXDato = Dato2CentrX(data, distancias)
    Pertenencias = []
    for element in CentroideXDato:
        Pertenencias.append(element + 1)
    Pertenencia = np.array(Pertenencias)
    return(Pertenencia)

### Resto de pasos de K-means común

Luego de generar los centroides iniciales de una manera distinta, seguimos el resto de los pasos del método de K-means, hasta obtener los centroides finales y los datos agrupados en base a estos:

In [ ]:
distancias = distACentrsXpunto(Centroides, puntos)

In [ ]:
CentroideXDato = Dato2CentrX(puntos, distancias)

In [ ]:
CentroideXDato = np.array(CentroideXDato)

In [ ]:
PuntosDCentr = AgrupDatos(Centroides, CentroideXDato, puntos)

In [ ]:
Centroides = Centrs_promedio(PuntosDCentr)

In [ ]:
Centroides_anterior = 0

while Centroides_anterior != Centroides:
    Centroides_anterior = Centroides
    distancias = distACentrsXpunto(Centroides_anterior, puntos)
    CentroideXDato = Dato2CentrX(puntos, distancias)
    CentroideXDato = np.array(CentroideXDato)
    PuntosDCentr = AgrupDatos(Centroides_anterior, CentroideXDato)
    Centroides = Centrs_promedio(PuntosDCentr)
    Xp = []
    Yp = []
    for element in Centroides:
        Xp.append(element[0])
        Yp.append(element[1])

    Colors = ['yellow', 'orange', 'lightblue', 'purple', 'lightgreen']

    for idx, Puntos in enumerate(PuntosDCentr):
        XColor = []
        YColor = []
        for element in Puntos:
            XColor.append(element[0])
            YColor.append(element[1])
        plt.scatter(XColor, YColor, color = Colors[idx], label = 'Datos de Centroide ' + str(idx + 1))

    plt.scatter(Xp, Yp, color = 'red', label = 'Centroides')
    plt.title('Ingreso Anual (k$) vs Edad')
    plt.xlabel('Ingreso Anual (k$)')
    plt.ylabel('Edad')
    plt.legend()
    display.clear_output(wait=True)
    plt.pause(1)

plt.show()

In [ ]:
for idx, Centroide in enumerate(Centroides):
    print('El centroide', str(idx + 1), 'tiene coordenadas', str(Centroide) + '.')

### Calculos utilizando librería de scikit learn con K-means++

Obtenemos los datos de k-means++

In [ ]:
puntos = data2point(X, Y)

kmeans = KMeans(n_clusters=5, init='k-means++', n_init=1, random_state=0).fit(puntos)
CentroideXDato = kmeans.labels_
Centroides = kmeans.cluster_centers_
PuntosDCentr = AgrupDatos(Centroides, CentroideXDato, puntos)

In [ ]:
Xp = []
Yp = []
for element in Centroides:
    Xp.append(element[0])
    Yp.append(element[1])

Colors = ['yellow', 'orange', 'lightblue', 'purple', 'lightgreen']

for idx, Puntos in enumerate(PuntosDCentr):
    XColor = []
    YColor = []
    for element in Puntos:
        XColor.append(element[0])
        YColor.append(element[1])
    plt.scatter(XColor, YColor, color = Colors[idx], label = 'Datos de Centroide ' + str(idx + 1))
    
plt.scatter(Xp, Yp, color = 'red', label = 'Centroides')
plt.title('Ingreso Anual (k$) vs Edad')
plt.xlabel('Ingreso Anual (k$)')
plt.ylabel('Edad')
plt.legend()
plt.show()

De igual manera que con el método de K-mean común, los resultados son similares, sin embargo se puede notar una aproximación más precisa e incluso más exacta al ejecutar los métodos repetidamente.

In [ ]:
for idx, Centroide in enumerate(Centroides):
    print('El centroide', str(idx + 1), 'tiene coordenadas', str(Centroide) + '.')